##**Import Dependencies**

In [1]:
import torch
from torch import nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

###**Device Configuration**

In [2]:
# Check for CUDA availability
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


##**Data Collection & Preprocessing**

In [13]:
# Load the dataset
data = load_breast_cancer()

In [14]:
X = data.data
y = data.target

In [15]:
# Splitting dataset into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [16]:
print(X.shape)
print(X_train.shape)
print(X_test.shape)

(569, 30)
(455, 30)
(114, 30)


In [17]:
# Standardizing the data using StandardScaler()
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [18]:
# Convert data to pytorch tensors
X_train = torch.tensor(X_train, dtype = torch.float32).to(device)
y_train = torch.tensor(y_train, dtype = torch.float32).to(device)
X_test = torch.tensor(X_test, dtype = torch.float32).to(device)
y_test = torch.tensor(y_test, dtype = torch.float32).to(device)

##**Neural Network Architecture**

In [27]:
class NeuralNet(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super().__init__()
    self.fc1 = nn.Linear(in_features = input_size, out_features = hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(in_features = hidden_size, out_features = output_size)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    out = self.sigmoid(out)

    return out

In [28]:
# Define Hyperparameters
input_size = X_train.shape[1]
hidden_size = 64
output_size = 1
learning_rate = 0.001
num_epochs = 100

In [29]:
# Intialize the model
model = NeuralNet(input_size, hidden_size, output_size).to(device)

In [30]:
model

NeuralNet(
  (fc1): Linear(in_features=30, out_features=64, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [31]:
# Define Loss and Optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(params=model.parameters(),
                             lr = learning_rate)

##**Training Neural Network**

In [35]:
for epoch in range(num_epochs):
  model.train()
  optimizer.zero_grad()
  outputs = model(X_train)
  loss = criterion(outputs, y_train.view(-1, 1))
  loss.backward()
  optimizer.step()

  # Calculate Accuracy
  with torch.no_grad():
    predicted = outputs.round()
    correct = (predicted == y_train.view(-1, 1)).float().sum()
    accuracy = correct/y_train.size(0)

  if (epoch+1) % 10 == 0:
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss : {loss.item():.4f}, Accuracy: {accuracy.item() * 100:.2f}%')

Epoch [10/100], Loss : 0.3461, Accuracy: 95.16%
Epoch [20/100], Loss : 0.2771, Accuracy: 96.26%
Epoch [30/100], Loss : 0.2280, Accuracy: 96.04%
Epoch [40/100], Loss : 0.1929, Accuracy: 96.48%
Epoch [50/100], Loss : 0.1674, Accuracy: 96.48%
Epoch [60/100], Loss : 0.1484, Accuracy: 96.92%
Epoch [70/100], Loss : 0.1337, Accuracy: 96.70%
Epoch [80/100], Loss : 0.1221, Accuracy: 97.14%
Epoch [90/100], Loss : 0.1125, Accuracy: 97.36%
Epoch [100/100], Loss : 0.1045, Accuracy: 97.36%


##**Model Evaluation**

In [37]:
# Evaluation on Training Data
model.eval()
with torch.no_grad():
  outputs = model(X_train)
  predicted = outputs.round()
  correct = (predicted == y_train.view(-1, 1)).float().sum()
  accuracy = correct / y_train.size(0)
  print(f'Accuracy on training data: {accuracy.item() * 100:.2f}%')

Accuracy on training data: 97.36%


In [39]:
# Evaluation on Testing data
model.eval()
with torch.no_grad():
  outputs = model(X_test)
  predicted = outputs.round()
  correct = (predicted == y_test.view(-1, 1)).float().sum()
  accuracy = correct / y_test.size(0)
  print(f'Accuracy on training data: {accuracy.item() * 100:.2f}%')

Accuracy on training data: 97.37%
